Sliding Puzzle with Breadth First Search and Depth First Search
=============
Nick Creel - April 17, 2020 - MIT License

I understand how Breadth and Depth first search work, but I do not understand exactly what kind of strategy is needed to solve a sliding puzzle. I didn't think this was the kind of confusoin where I should ask for clarification, so I did not do so...

I know that the puzzle must end up in row-major order... From watching [this gif](https://en.wikipedia.org/wiki/Sliding_puzzle#/media/File:Batgirl.gif), it seems like a good strategy might be to get the smallest value to root position and then begin rearranging from the top again. This would necessarily require moving the "empty" square to the top, then percolating it back down again. I also noticed that, in the solved puzzle, the left child's value is the value of the parent added to three, while the right child's value is the value of the parent added to one. 

I attempted to work out a smaller case by hand using a depth first search and b:

```
DFS start: (asterisk indicates where we've traveled. first we search for empty and propogate up.

    2 *
   / \
  1   3
   \ /
    .
    
    2 
   / \
* 1   3
   \ /
    .

    2 
   / \
  1   3   # found empty
   \ /
  * .
  
    2 
   / \
 *1   3   # return back up to one
   \ /
    .
    
    2 
   / \
 *.   3   # swap
   \ /
    1
    
   *2 
   / \
  .   3   # return back up to 2
   \ /
    1

   *. 
   / \
  2   3   # swap
   \ /
    1

    . 
   / \
  2   3*  # now go down right
   \ /
    1
    
    3 
   / \
  2   .*  # if parent = empty, swap
   \ /
    1
  
    3 
   / \
  2   .  # go down
   \ /
    1*

    3 
   / \
  2   1  # swap
   \ /
    .*
    
    ... go back up to top once empty has percolated back down? essentially start search over.
    
    3 *
   / \
  2   1  
   \ /
    .
    
    
    3 
   / \
 *2   1  
   \ /
    .
   
    3 
   / \
  2   1  
   \ /
    .*
   
    3 
   / \
  2*  1  
   \ /
    .
    
    3 
   / \
  .*  1  # swap
   \ /
    2
    
    3 *
   / \
  .   1  
   \ /
    2
    
    .*
   / \
  3   1 # swap
   \ /
    2
    
        
    .
   / \
  3   1* # go down right
   \ /
    2
    
    1    
   / \
  3   .* # swap
   \ /
    2
    
    1    
   / \
  3   . 
   \ /
    2*
    
        
    1    
   / \
  3   2 #swap
   \ /
    .*
    
    and then check if we're done somehow with another search.  
```

When writing the code for breadth first and depth first search, I consulted chapter 5 of Skiena's Algorithm Design Manual, which covers both BFS and DFS, as well as [Jim's class notes on trees.](https://cs.marlboro.college/cours/spring2019/algorithms/code/graphs/search/tree.py)

In [87]:
class Queue:
    # for BFS
    # fifo
    def __init__(self):
        self.queue = []
    def enqueue(self, value):
        self.queue.append(value)
    def dequeue(self):
        return self.queue.pop(0)
    def peek(self):
        return self.queue[0]
    def isEmpty(self):
        if len(self.queue) == 0:
            return True
        else:
            return False

class Stack:
    # for DFS
    # lifo
    def __init__(self):
        self.stack = []
    def push(self,value):
        self.stack.append(value)
    def pop(self):
        return self.stack.pop()
    def isEmpty(self):
        if len(self.stack) == 0:
            return True
        else: 
            return False
        
class Node:
    def __init__(self, value, left=None, right=None, discovered = False, root=False):
        self.value = value
        self.left = left
        self.right = right
        self.discovered = discovered
        self.root = root
    def isSolved(self, puzzleMax):
        if self.value == None and self.left == None and self.right == None:
            return True
        elif self.root == True and self.value != None and self.value == self.left.value + 3 and self.right.value + 1: 
            # this only works for root
            return True
        elif self.root == False and self.value != None and self.left != None and self.value == self.left.value + 3:
            return True
        elif self.root == False and self.value != None and self.right != None and self.value == self.right.value + 1:
            return True
        else:
            return False

# looks like 
#          2
#   left--/ \--right
#        5   1 
#       / \ / \
#      7   4   3
#      \ /  \ /  
#       8    6
#        \  /
#        empty
#
# depth first would go straight down
# breadth first would go down + right, down + right...
# we want it to look like this
#          1
#   left--/ \--right
#        4   2 
#       / \ / \
#      7   5   3
#      \ /  \ /  
#       8    6
#        \  /
#        empty
# binary tree
# finish conditions:
#   left = self + 3
#   right = self + 1
# & pieces can only be moved to empty spaces 
# that are immediately left or right
# does it make sense to propogate empty upwards?
# we can move the empty square around like a piece, propogate it to the top, then send it back down...
# we know that the empty piece must have the max number and the max number - 2 as parents. 
# but this must true alongside the fact that all nodes (beside the max and max - 2) have left = self+3 and
# right = self +1.  

In [88]:
## make the tree
# looks like 
#          2
#   left--/ \--right
#        5   1 
#       / \ / \
#      7   4   3
#      \ /  \ /  
#       8    6
#        \  /
#        empty

## two is the root, no parent

## actually i made a smaller tree
##      2
##     / \
##    1   3
##     \ /
##      .

# read skiena's algorithm manual chapter 5 on BFS and DFS to review. 

one = Node(1)
two = Node(2)
three = Node(3)
empty = Node(None)

one.right = empty
two.left = one
two.right = three
three.left = empty

## even though these two functions share largely the same structure, 
## the queue and stack ultimately determine whether or not the search is B or D first.

def depthFirstSearch(node):
    stack = Stack()
    stack.push(node)
    while stack.isEmpty() == False:
        current = stack.pop()
        current.discovered = True
        print(current.value)
        if current.right != None and current.right.discovered == False:
            current.right.discovered = True
            stack.push(current.right)
        if current.left != None and current.left.discovered == False:
            current.left.discovered = True
            stack.push(current.left)

def breadthFirstSearch(node):
    queue = Queue()
    queue.enqueue(node)
    while queue.isEmpty() == False:
        current = queue.dequeue()
        print(current.value)
        print(current.isSolved())
        if current.left != None:
            print("left is not None")
            queue.enqueue(current.left)
        if current.right != None:
            print("right is not none")
            queue.enqueue(current.right)
        if current.value == None and queue.isEmpty() == False:
            print("current.value == None")
            above = queue.dequeue()
            if above.left.value == None:
                temp = current.left
                current.left = above.left
                above.left = current.left
                queue.enqueue(current)
                queue.enqueue(above)
            elif above.right.value == None:
                temp = current.right
                current.left = above.right
                above.left = current.right
                queue.enqueue(current)
                queue.enqueue(above)
                
                



In [89]:
breadthFirstSearch(two)

2
False
left is not None
right is not none
1


TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'

In [68]:
depthFirstSearch(two)

2
1
None
3
